Test other .py code

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import logging
# import sys
import subprocess
logging.basicConfig(level=logging.DEBUG)

# from setup_logger import setup_log
# setup_log('test.log', mode='w')

# Test get dosage
from load_dosage import get_dosage_of_snps_from_file, get_dosage_matrix

## 1. Test load_dosage.py (done)

In [23]:
# Test get dosage
vcf_path = '/data100t1/home/wanying/CCHC/lipidomics/input_docs/lipidomic_sample_vcfs/202312_redo_training_vcfs'
vcf_fn = 'max_unrelated_set_chr*.vcf.gz'
snp_fn = '/data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/outputs/fastGWA/lipid_class_filter_by_pval_1e-05_maf_0.01/AC-OH_SNPs.pval_1e-05.maf_0.01.txt'
df_dosage = get_dosage_of_snps_from_file(vcf_path, vcf_fn, snp_fn, chr_col='CHR', pos_col='POS', drop_multiallelic=True)

# - Getting dosage from chr1: 100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  5.56it/s]
root - INFO - # CHRchr1: Number of multiallelic sites dropped=0; Number of SNPs not found=0
# - Getting dosage from chr2: 100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.59it/s]
root - INFO - # CHRchr2: Number of multiallelic sites dropped=0; Number of SNPs not found=0
# - Getting dosage from chr4: 100%|█████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00, 11.53it/s]
root - INFO - # CHRchr4: Number of multiallelic sites dropped=0; Number of SNPs not found=0
# - Getting dosage from chr5: 100%|█████████████████████████████████████████████████████████████████████████████████| 81/81 [00:05<00:00, 15.55it/s]
root - INFO - # CHRchr5: Number of multiallelic sites dropped=3; Number of SNPs not found=0
# - Getting dosage from chr6: 100%|█

In [2]:
d = get_dosage_of_snps_from_file('/data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/inputs/genotype_dosage/train/lipid_species',
                             'lipid_species_chr*.pval_0.001_maf_0.05.vcf.gz',
                             '/data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/outputs/fastGWA/lipid_species_filter_by_pval_1e-05/PC-44:5-_SNPs.pval_1e-05.txt',
                             chr_col='CHR', pos_col='POS', drop_multiallelic=True)


# - Getting dosage from chr22: 100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.37it/s]


# 2. Create commands to run all traits

In [ ]:
'''
lip_type=species
lipid="PC(28:0)"
python run_ml_model.py \
--output_prefix "${lipid}" \
--output_dir /data100t1/home/wanying/CCHC/lipidomics/20240405_rerun_fix_pearson_r_bug/output/${lip_type} \
--train_vcf_path /data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/inputs/genotype_dosage/train/lipid_${lip_type} \
--train_vcf_fn lipid_${lip_type}_chr*.pval_0.001_maf_0.05.vcf.gz \
--test_vcf_path /data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/inputs/genotype_dosage/test/lipid_${lip_type} \
--test_vcf_fn lipid_${lip_type}_chr*.pval_0.001_maf_0.05.test.vcf.gz \
--snp_fn /data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/outputs/fastGWA/lipid_species_filter_by_pval_1e-05/PC-28:0-_SNPs.pval_1e-05.txt \
--trait_name ${lipid} \
--trait_fn /data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/inputs/lipid_trait/lipid_${lip_type}_ID_matched.no_dup.residual.all.txt \
--n_alphas 100 \
--drop_multiallelic True \
--reg_type elastic_net \
--id_col genotype_id
'''

In [13]:
# Load lipid species to run
lip_fn = '/data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/inputs/lipid_trait/lipid_species.list'
df_lip = pd.read_csv(lip_fn, sep='\t', header=None)
lip_type = 'species'
path_code = '/data100t1/home/wanying/CCHC/lipidomics/20240405_rerun_fix_pearson_r_bug/code'
fh = open('./slurm/lip_species.txt', 'w')

for lip in df_lip[0]:
    if '/' in lip: prefix = lip.replace('/', '_') # To avoid / in output file name
    else: prefix = lip
        
    modified_lip_name = lip.replace('/', '-').replace('(','-').replace(')','-').replace(' ','_') # Replace special character
    cmd = f'lip_type=species;lipid="{lip}";python {path_code}/run_ml_model.py '
    cmd += f'--output_prefix "{prefix}" '
    cmd += '--output_dir /data100t1/home/wanying/CCHC/lipidomics/20240405_rerun_fix_pearson_r_bug/output/${lip_type} '
    cmd += '--train_vcf_path /data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/inputs/genotype_dosage/train/lipid_${lip_type} '
    cmd += '--train_vcf_fn lipid_${lip_type}_chr*.pval_0.001_maf_0.05.vcf.gz '
    cmd += '--test_vcf_path /data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/inputs/genotype_dosage/test/lipid_${lip_type} '
    cmd += '--test_vcf_fn lipid_${lip_type}_chr*.pval_0.001_maf_0.05.test.vcf.gz '
    cmd += f'--snp_fn /data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/outputs/fastGWA/lipid_species_filter_by_pval_1e-05/{modified_lip_name}_SNPs.pval_1e-05.txt '
    cmd += '--trait_name "${lipid}" '
    cmd += '--trait_fn /data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/inputs/lipid_trait/lipid_${lip_type}_ID_matched.no_dup.residual.all.txt '
    cmd += '--n_alphas 100 --drop_multiallelic True --reg_type elastic_net --id_col genotype_id --overwrite False'
    fh.write(cmd+'\n')
fh.close()

In [19]:
# Load lipid class to run
lip_fn = '/data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/inputs/lipid_trait/lipid_class.list'
df_lip = pd.read_csv(lip_fn, sep='\t', header=None)
lip_type = 'class'
path_code = '/data100t1/home/wanying/CCHC/lipidomics/20240405_rerun_fix_pearson_r_bug/code'
fh = open('./slurm/lip_class.txt', 'w')

for lip in df_lip[0]:
    modified_lip_name = lip.replace('/', '-').replace('(','-').replace(')','-').replace(' ','_') # Replace special character
    cmd = f'lip_type=class;lipid="{lip}";python {path_code}/run_ml_model.py '
    cmd += '--output_prefix "${lipid}" --output_dir /data100t1/home/wanying/CCHC/lipidomics/20240405_rerun_fix_pearson_r_bug/output/${lip_type} '
    cmd += '--train_vcf_path /data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/inputs/genotype_dosage/train/lipid_${lip_type} '
    cmd += '--train_vcf_fn lipid_${lip_type}_chr*.pval_0.001_maf_0.05.vcf.gz '
    cmd += '--test_vcf_path /data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/inputs/genotype_dosage/test/lipid_${lip_type} '
    cmd += '--test_vcf_fn lipid_${lip_type}_chr*.pval_0.001_maf_0.05.test.vcf.gz '
    cmd += f'--snp_fn /data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/outputs/fastGWA/lipid_class_filter_by_pval_1e-05/{modified_lip_name}_SNPs.pval_1e-05.txt '
    cmd += '--trait_name "${lipid}" '
    cmd += '--trait_fn /data100t1/home/wanying/CCHC/lipidomics/20231211_rerun/inputs/lipid_trait/lipid_${lip_type}_ID_matched.no_dup.residual.all.txt '
    cmd += '--n_alphas 100 --drop_multiallelic True --reg_type elastic_net --id_col genotype_id --overwrite False'
    fh.write(cmd+'\n')
fh.close()